In [1]:
import scipy.io
import os
import numpy as np
from numpy import arange
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
stride = 1

def tranform_to_sampel(raw,new):
    num = 0
    for i in arange(0,s1,stride):
        if i+n1>=s1:
            l1 = list(arange(i,s1))
            l2 = list(arange(0,n1-len(l1)))
            il = l1+l2
        else:
            il = list(arange(i,i+n1))
        for j in range(0,s2,stride):
            if j+n2>=s2:
                l1 = list(arange(j,s2))
                l2 = list(arange(0,n2-len(l1)))
                jl = l1+l2
            else:
                jl =list(arange(j,j+n2)) 
                
            new[num,:,:,:]= raw[il,:,:][:,jl,:]
            if num == 0:
                print(il,jl)            
            num = num + 1

    return new

def tranform_to_target(raw,new,out):
    num = 0
    for i in arange(0,s1,stride):
        if i+n1>=s1:
            l1 = list(arange(i,s1))
            l2 = list(arange(0,n1-len(l1)))
            il = l1+l2
        else:
            il = list(arange(i,i+n1))
        for j in range(0,s2,stride):
            if j+n2>=s2:
                l1 = list(arange(j,s2))
                l2 = list(arange(0,n2-len(l1)))
                jl = l1+l2
            else:
                jl =list(arange(j,j+n2)) 
                
            new[num,:,:,:]= raw[il,:,:][:,jl,:]
            out[num,:] = new[num,:,:,:][3,3,:]
            if num == 0:
                print(il,jl)            
            num = num + 1

    return out


In [4]:
input_u = scipy.io.loadmat("u_F_xyz_T1.mat", mdict=None, appendmat=True)
input_v = scipy.io.loadmat("v_F_xyz_T1.mat", mdict=None, appendmat=True)
input_w = scipy.io.loadmat("w_F_xyz_T1.mat", mdict=None, appendmat=True)
input_tau11 = scipy.io.loadmat("tau11_xyz_T1.mat", mdict=None, appendmat=True)
input_tau12 = scipy.io.loadmat("tau12_xyz_T1.mat", mdict=None, appendmat=True)
input_tau13 = scipy.io.loadmat("tau13_xyz_T1.mat", mdict=None, appendmat=True)
input_tau22 = scipy.io.loadmat("tau22_xyz_T1.mat", mdict=None, appendmat=True)
input_tau23 = scipy.io.loadmat("tau23_xyz_T1.mat", mdict=None, appendmat=True)
input_tau33 = scipy.io.loadmat("tau33_xyz_T1.mat", mdict=None, appendmat=True)

In [5]:
u_F = input_u['u_F']
v_F = input_v['v_F']
w_F = input_w['w_F']
tau11 = input_tau11['tau11']
tau12 = input_tau12['tau12']
tau13 = input_tau13['tau13']
tau22 = input_tau22['tau22']
tau23 = input_tau23['tau23']
tau33 = input_tau33['tau33']

In [6]:
s1,s2,s3 = tau11.shape
n1 = 7
n2 = 7
n3 = s3

stride = 1
num = 0
for i in arange(0,s1,stride):
    for j in range(0,s2,stride):
        num = num +1

ini = np.zeros((num,n1,n2,n3))
newy = np.zeros((num,n3))
#transform into small box
vf = np.float32(tranform_to_sampel(v_F,ini))
uf = np.float32(tranform_to_sampel(u_F,ini))
wf = np.float32(tranform_to_sampel(w_F,ini))

y11 = np.float32(tranform_to_target(tau11,ini,newy))

y12= np.float32(tranform_to_target(tau12,ini,newy))
y13= np.float32(tranform_to_target(tau13,ini,newy))
y22= np.float32(tranform_to_target(tau22,ini,newy))
y23= np.float32(tranform_to_target(tau23,ini,newy))
y33= np.float32(tranform_to_target(tau33,ini,newy))

[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]


In [7]:
ytrain11 = np.reshape(y11, (y11.shape[0], -1))
batch_size = 128
nb_epoch = 250

#input_shape = (n1,n2,n3)
kernel_size = [3, 3]
pool_size = [2, 2]
outsize = n3
nb_filters = 16

In [8]:
def cnn_model_fn(features, labels, mode):
    #first model
    visible1 = tf.reshape(features['u'], [-1, 7, 7, 75])
    ##Input(shape=(vf.shape[1],vf.shape[2],vf.shape[3]))
    conv11 = tf.layers.conv2d(inputs=visible1, filters=nb_filters, kernel_size=kernel_size, 
                            padding="valid", activation=tf.nn.relu)
    conv12 = tf.layers.conv2d(inputs=conv11, filters=nb_filters, kernel_size=kernel_size, 
                            padding='valid', activation=tf.nn.relu)
    pool12 = tf.layers.max_pooling2d(inputs=conv12, pool_size=pool_size, strides=2)
    flat1 = tf.layers.flatten(inputs=pool12)
    drop1 = tf.layers.dropout(inputs=flat1, rate=0.2)
    
    visible2 = tf.reshape(features['v'], [-1, 7, 7, 75])
    #Input(shape=(vf.shape[1],vf.shape[2],vf.shape[3]))
    conv21 = tf.layers.conv2d(inputs=visible2, filters=nb_filters, kernel_size=kernel_size, 
                            padding='valid', activation=tf.nn.relu)
    conv22 = tf.layers.conv2d(inputs=conv21, filters=nb_filters, kernel_size=kernel_size, 
                            padding='valid', activation=tf.nn.relu)
    pool22 = tf.layers.max_pooling2d(inputs=conv22, pool_size=pool_size, strides=2)
    flat2 = tf.layers.flatten(inputs=pool22)
    drop2 = tf.layers.dropout(inputs=flat2, rate=0.2)
    
    visible3 = tf.reshape(features['w'], [-1, 7, 7, 75])
    #Input(shape=(vf.shape[1],vf.shape[2],vf.shape[3]))
    conv31 = tf.layers.conv2d(inputs=visible3, filters=nb_filters, kernel_size=kernel_size, 
                            padding='valid', activation=tf.nn.relu)
    conv32 = tf.layers.conv2d(inputs=conv31, filters=nb_filters, kernel_size=kernel_size,
                            padding='valid', activation=tf.nn.relu)
    pool32 = tf.layers.max_pooling2d(inputs=conv32, pool_size=pool_size, strides=2)
    flat3 = tf.layers.flatten(inputs=pool32)
    drop3 = tf.layers.dropout(inputs=flat3, rate=0.2)
    
    merge = tf.concat([drop1, drop2,drop3], 1)
    hidden1 = tf.layers.dense(inputs=merge, units=64, activation=tf.nn.relu)
    hiddrop = tf.layers.dropout(inputs=hidden1, rate=0.3)
    hidden2 = tf.layers.dense(inputs=hiddrop, units=128, activation=tf.nn.relu)
    y_pred = tf.layers.dense(inputs=hidden2, units=outsize, activation=tf.nn.relu)
        # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.mean_squared_error(labels, y_pred)
    
    predictions = {
        "matrix": y_pred,
        "loss": tf.metrics.mean_squared_error(labels, y_pred, name="mse_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    for var in tf.trainable_variables():
        tf.summary.histogram(var.name, var)
    # Summarize all gradients
    for grad, var in grads:
        tf.summary.histogram(var.name + '/gradient', grad)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()
    summary_hook = tf.train.SummarySaverHook(
        save_steps=100,
        summary_op=merged_summary_op)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks=[summary_hook])
    
    eval_metric_ops = {
        "mse": tf.metrics.mean_absolute_error(labels=labels, predictions=y_pred)
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [9]:
nn = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="./cnn_model_log")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"loss": "mse_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
    
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'u': uf, 'v': vf, 'w': wf},
    y=ytrain11,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_service': None, '_model_dir': './cnn_model_log', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C44AD4A2E8>, '_tf_random_seed': None, '_is_chief': True, '_master': '', '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_task_type': 'worker', '_num_ps_replicas': 0}


In [10]:
nn.train(input_fn=train_input_fn,steps=1500)

INFO:tensorflow:Summary name conv2d/kernel:0 is illegal; using conv2d/kernel_0 instead.
INFO:tensorflow:Summary name conv2d/bias:0 is illegal; using conv2d/bias_0 instead.
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name conv2d_2/kernel:0 is illegal; using conv2d_2/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_2/bias:0 is illegal; using conv2d_2/bias_0 instead.
INFO:tensorflow:Summary name conv2d_3/kernel:0 is illegal; using conv2d_3/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_3/bias:0 is illegal; using conv2d_3/bias_0 instead.
INFO:tensorflow:Summary name conv2d_4/kernel:0 is illegal; using conv2d_4/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_4/bias:0 is illegal; using conv2d_4/bias_0 instead.
INFO:tensorflow:Summary name conv2d_5/kernel:0 is illegal; using conv2d_5/kernel_0 instead.
INFO:tensorflow: